Résumé du Notebook : Agent Conversationnel Touristique des Alpes-Maritimes (RAG)

Ce notebook détaille les étapes de création d'un agent conversationnel innovant, basé sur un système RAG (Retrieval-Augmented Generation), dédié à l'exploration et à la découverte des Alpes-Maritimes. L'objectif est de fournir aux utilisateurs un outil interactif pour planifier leurs voyages, obtenir des informations précises et personnalisées sur les attractions touristiques, les activités, les hébergements et les événements locaux.

Le notebook couvre :

La collecte et le prétraitement d'une base de connaissances riche et diversifiée sur les Alpes-Maritimes.

La sélection et la configuration d'un modèle de langage (LLM) puissant, adapté aux spécificités du domaine touristique.

L'implémentation d'un système de récupération d'informations performant, capable d'extraire les données pertinentes en fonction des requêtes des utilisateurs.

L'intégration du RAG pour générer des réponses informatives, contextuelles et engageantes.

La création d'une interface utilisateur conversationnelle conviviale, facilitant l'interaction avec l'agent.

L'optimisation du système à travers des tests rigoureux et des ajustements basés sur les retours utilisateur

# Installation des dépendances 
#pdfplumber, pour parser les documents pdf,
1- poetry add pdfplumber
#sentence-transformers pour les embeddings 
2- poetry  sentence_transformers

# # l'API OpenAI pour générer des réponses augmentées par le contexte.
 poetry add openai==0.28

In [1]:
## importation des pakages 

In [2]:
from sentence_transformers import SentenceTransformer, util
import os
import pdfplumber
from langchain.text_splitter import RecursiveCharacterTextSplitter

## importation des modules spécifiques

import utils as u
#from utils import RecursiveCharacterTextSplitter


/home/beatrice/.cache/pypoetry/virtualenvs/llms-IHN9N-5k-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Étape 1 : Extraction et segmentation du texte
Dans cette première étape, nous allons nous concentrer sur l'extraction du texte d'un fichier PDF et sa segmentation en blocs de taille définie. L'objectif est de rendre le texte extrait plus facile à traiter dans les phases suivantes.

 ### Détails :

* Extraction du texte : À l'aide de pdfplumber, vous allez extraire tout le texte d'un fichier PDF donné. Cette étape vous permettra d'obtenir une version brute du contenu du document, qui servira de base pour les autres étapes.

* Segmentation du texte : Une fois le texte extrait, vous le diviserez en segments de taille fixe (par exemple, 500 caractères par segment) à l'aide de la fonction textwrap.wrap() de Python. Cela permet de découper le texte en morceaux plus faciles à manipuler dans la phase suivante où nous calculerons les similarités.

**Cette étape est cruciale, car elle permet de structurer le texte pour le rendre exploitable par la suite. La qualité de l'extraction et de la segmentation influencera directement les résultats des phases suivantes.**

In [8]:
def extract_text_and_tables(
    directory_path: str,
    chunk_size: int = 1000,
    chunk_overlap: int = 200
) -> list:
    """
    Parcourt un répertoire et renvoie une seule liste de chaînes de caractères.
    Cette liste contient, pêle-mêle :
      - Les chunks de texte issus de chaque page PDF,
      - Les chunks de texte issus de chaque table.

    Pour les tables, on les transforme en texte en
    séparant les cellules d'une ligne par "|" et
    les lignes entre elles par des sauts de ligne.
    """

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )

    results = []  # Une seule liste de strings

    for filename in os.listdir(directory_path):
        if filename.lower().endswith(".pdf"):
            pdf_path = os.path.join(directory_path, filename)

            with pdfplumber.open(pdf_path) as pdf:
                for page in pdf.pages:
                    # --- 1) Texte normal ---
                    raw_text = page.extract_text() or ""
                    raw_text = raw_text.strip()

                    if raw_text:
                        chunks = text_splitter.split_text(raw_text)
                        results.extend(chunks)

                    # --- 2) Tables converties en texte ---
                    page_tables = page.extract_tables() or []
                    for table in page_tables:
                        if not table:
                            continue

                        # Construit une représentation texte de la table
                        # exemple : on sépare les cellules par " | "
                        #          et on sépare les lignes par "\n"
                        lines = []
                        for row in table:
                            # row est une liste de cellules
                            # on ignore le cas None pour éviter les erreurs de jointure
                            cleaned_row = [cell if cell else "" for cell in row]
                            line = " | ".join(cleaned_row)
                            lines.append(line)

                        table_text = "\n".join(lines).strip()

                        # Puis, on chunk ce texte comme n'importe quel texte
                        if table_text:
                            table_chunks = text_splitter.split_text(table_text)
                            results.extend(table_chunks)

    return results


In [9]:
doc_list = extract_text_and_tables(
    directory_path="documents",
    chunk_size=500,
    chunk_overlap=200
)
doc_list[10]

'SOMMAIRE ÉDITO\nPage Page Page\n14 28\n02 La Métropole\nLE MOYEN-PAYS LE HAUT-PAYS\nNice Côte d’Azur\n16 Aspremont 31 Les stations de montagne Le guide que vous tenez entre les mains, vous ouvre les portes d’un univers\n05 Édito\n17 Bonson & stations de ski\ndont une vie ne suffirait pas à explorer tous les trésors tant ils sont nombreux !\nPeu de territoires dans le monde peuvent offrir une telle variété de\nCarros 32 Bairols\n06 paysages, une telle densité de sites remarquables, autant d’événements,'

## Étape 2 : Calcul des Embeddings et recherche des segments similaires
Dans cette deuxième étape, nous allons calculer des embeddings pour chaque segment de texte extrait et effectuer une recherche pour identifier les segments les plus similaires à une requête utilisateur donnée.

### Détails :

* Calcul des embeddings : Nous allons utiliser la bibliothèque sentence-transformers pour générer des vecteurs d’embeddings pour chaque segment de texte. Ces vecteurs capturent les caractéristiques sémantiques des segments, permettant ainsi de comparer leur pertinence par rapport à une requête donnée.

* Recherche des segments les plus similaires : Une fois les embeddings calculés, nous allons utiliser la similarité cosinus pour mesurer la proximité entre l’embedding de la requête utilisateur et les embeddings des segments de texte. Les segments les plus proches seront sélectionnés pour la prochaine étape.

* Sélection des top-N segments : Nous sélectionnerons les n segments les plus similaires à la requête, qui serviront de contexte pour générer une réponse augmentée dans la phase suivante.

**Cette étape est essentielle pour filtrer le contenu extrait en fonction de sa pertinence par rapport à la requête utilisateur. Elle permet de s'assurer que seuls les segments les plus utiles sont utilisés pour générer une réponse cohérente et contextuelle**

In [10]:
from sentence_transformers import SentenceTransformer, util

# Charger un modèle pré-entraîné de sentence-transformers
model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
from sentence_transformers import SentenceTransformer, util

# Charger un modèle pré-entraîné de sentence-transformers
model = SentenceTransformer('all-MiniLM-L6-v2')
# Calcul des embeddings pour chaque segmnt de texte (tableaux NumPy par défaut)
embeddings = model.encode(doc_list)
# Requête utilisateur pour laquelle nous cherchons les segments similaires
query = "Quel sont les communes dans les alpes maritimes ?"
query_embedding = model.encode(query)

# Calcul des similarités cosinus entre la requête et les segments
similarities = util.cos_sim(query_embedding, embeddings)

# Sélection des top-N documents les plus similaires
top_n = 10
top_results = similarities.topk(top_n)[1]  # Indices des top-N documents similaires

# Accéder aux indices des résultats top-N
top_n_paragraphs = [doc_list[i] for i in top_results[0]]

# Affichage des top-N segments les plus similaires
for i, para in enumerate(top_n_paragraphs, 1):
    print(f"Segment {i} similaire : {para}\n")

In [11]:
# Calcul des embeddings pour chaque segmnt de texte (tableaux NumPy par défaut)
embeddings = model.encode(doc_list)

In [12]:
# Requête utilisateur pour laquelle nous cherchons les segments similaires
query = "Quel sont les communes dans les alpes maritimes ?"
query_embedding = model.encode(query)

# Calcul des similarités cosinus entre la requête et les segments
similarities = util.cos_sim(query_embedding, embeddings)

# Sélection des top-N documents les plus similaires
top_n = 10
top_results = similarities.topk(top_n)[1]  # Indices des top-N documents similaires

# Accéder aux indices des résultats top-N
top_n_paragraphs = [doc_list[i] for i in top_results[0]]

# Affichage des top-N segments les plus similaires
for i, para in enumerate(top_n_paragraphs, 1):
    print(f"Segment {i} similaire : {para}\n")

Segment 1 similaire : Roquebilière
Italie
Clans
Bairols Alpes
de Haute
Provence
Lantosque
La Tour-sur-Tinée
Alpes
Maritimes
LE MOYEN-PAYS
Tournefort
Var
Mer Méditerranée
Utelle
Non loin des rivages de la
Méditerranée, ce territoire offre
un havre de calme, fraîcheur et
sérénité. Les villages, blottis dans
Bonson
Duranus des écrins de verdure ou accrochés à la
falaise tels des nids d’aigle, proposent au
visiteur de flâner dans leurs charmantes
Gilette Levens
ruelles. Il y découvrira un patrimoine

Segment 2 similaire : Par décret n° 2003-1169 du 2 décembre 2003 le Gouvernement a approuvé la directive
territoriale d'aménagement des Alpes-Maritimes (Journal Officiel du 9 décembre 2003).
, est située dans le secteur Haut-
Pays.
La DTA précise que le Haut-Pays présente une richesse patrimoniale exceptionnelle
convient de préserver et de valoriser. Cette richesse, qui contribue à de qualité de
la es loisirs.
-
préservation de la qualité patrimoniale, et de permettre un développement durable 

## Étape 3 : Génération de réponses avec ChatGPT
Dans cette troisième, nous allons utiliser les segments de texte sélectionnés et une requête utilisateur pour générer une réponse contextuelle en utilisant l'API OpenAI, avec un modèle de type ChatGPT.

### Objectifs :

* Utiliser le modèle GPT-4 pour générer une réponse basée sur les segments de texte les plus similaires à la requête.

* Envoyer les segments sélectionnés et la requête utilisateur sous forme de messages à l’API ChatGPT.
Obtenir une réponse contextuelle, augmentée par les informations pertinentes extraites du texte.

* Détails :

* Création du contexte : Nous allons concaténer les segments sélectionnés lors de l’étape précédente afin de créer un contexte cohérent à transmettre au modèle GPT-4. Ce contexte servira à fournir un maximum d’informations pertinentes pour générer une réponse précise.

* Appel à l’API ChatGPT : En utilisant le contexte et la requête de l’utilisateur, nous formulerons un prompt que nous enverrons à l’API ChatGPT. Le prompt sera structuré sous forme de messages (avec les rôles "system" et "user") pour que le modèle comprenne le contexte de la conversation.

* Génération et récupération de la réponse : Le modèle GPT-4 générera une réponse basée sur le contexte fourni. La réponse sera ensuite récupérée et affichée comme résultat final.

**Cette étape finalise le processus de RAG en combinant la recherche d’informations pertinentes dans un corpus de texte et la génération de réponses intelligentes basées sur ces informations.**

In [14]:

import openai


openai.api_key = "sk-proj-q24aiX6efr5M2SC5ySeVT3BlbkFJdlSFxh4bMKc6G2lL36t4"

# Concaténer les top-N segments en un seul contexte
context = "\n".join(top_n_paragraphs)
prompt = f"Contexte :\n{context}\n\nQuestion : {query}\nRéponse :"

# Appeler l'API d'OpenAI
response = openai.ChatCompletion.create(
    model="gpt-4o",  # Utilisation du modèle gpt-4o
    messages=[
        {"role": "system", "content": "Tu es un assistant specialisé dans la recherche d'information à partir de documents fournis. Tes réponses doivent absolument provenir du contexte fourni."},
        {"role": "user", "content": prompt}
    ],
    temperature=0
)

# Extraire et afficher la réponse générée
generated_response = response['choices'][0]['message']['content'].strip()
print(f"Réponse générée : {generated_response}")
     

Réponse générée : Les communes dans les Alpes-Maritimes mentionnées dans le contexte sont :

- Roquebilière
- Clans
- Bairols
- Lantosque
- La Tour-sur-Tinée
- Tournefort
- Utelle
- Bonson
- Duranus
- Gilette
- Levens
- La Roquette-sur-Var
- Saint-Martin-du-Var
- Saint-Blaise
- Le Broc
- Castagniers
- Châteauneuf-Villevielle
- Gattières
- Tourrette-Levens
- Carros
- Aspremont
- Colomars
- Saint-Jeannet
- Falicon
- Drap
- Saint-André-de-la-Roche
- Vence
- La Trinité
- Eze
- Cap d’Ail
- Nice
- Villefranche-sur-Mer
- La Gaude
- Beaulieu-sur-Mer
- Saint-Laurent-du-Var
- Saint-Dalmas-le-Selvage
- Saint-Etienne-de-Tinée
- Auron
- Isola
- Isola 2000
- Saint-Sauveur-sur-Tinée
- Roure
- Valdeblore
- Saint-Martin-Vésubie
- La Colmiane
- Roubion
- Rimplas
- Belvédère
- Venanson
- Ilonse


In [17]:
# Requête utilisateur pour laquelle nous cherchons les segments similaires
query = "Quel sont les lieux les plus visité ?"
query_embedding = model.encode(query)

# Calcul des similarités cosinus entre la requête et les segments
similarities = util.cos_sim(query_embedding, embeddings)

# Sélection des top-N documents les plus similaires
top_n = 10
top_results = similarities.topk(top_n)[1]  # Indices des top-N documents similaires

# Accéder aux indices des résultats top-N
top_n_paragraphs = [doc_list[i] for i in top_results[0]]

# Affichage des top-N segments les plus similaires
for i, para in enumerate(top_n_paragraphs, 1):
    print(f"Segment {i} similaire : {para}\n")

Segment 1 similaire : + + Néanmoins, | 
Voirie et trafic | 0 | et
.
Bâti | - | du chalet
Foncier | Acquisition de de
- emprises partielles
réaliser les travaux | ux emprises totales et de deux
de parcelles privées afin de pouvoir
.
Réseaux | 0 | 
Activités | Le projet, en améli
proposées sur le Fr
+ + touristique
pérennisation des a
commerciales de la | orant le fonctionnement des activités
ont de neige participe à attrait
et ainsi à la
ctivités économiques, touristiques et
station.
Paysage | L
+

Segment 2 similaire : St-André la Roche 39 St-Étienne-de-Tinée qui, en toute saison, frappent nos visiteurs et inspirent les plus grands artistes.
25 St-Blaise St-Martin-Vésubie
Bienvenue dans notre Métropole Nice Côte d’Azur ! Nos équipes des bureaux
d’informations touristiques vous attendent !
Saint-Jeannet 40 St-Sauveur sur tinée
26 St-Martin-du-Var Tournefort
Tourrette-Levens 41 Valdeblore
Bienvenue dans notre Métropole Nice Côte d’Azur !
27 Utelle Venanson
Vence
42 Infos pratiques Christ

In [18]:

import openai


openai.api_key = "sk-proj-q24aiX6efr5M2SC5ySeVT3BlbkFJdlSFxh4bMKc6G2lL36t4"

# Concaténer les top-N segments en un seul contexte
context = "\n".join(top_n_paragraphs)
prompt = f"Contexte :\n{context}\n\nQuestion : {query}\nRéponse :"

# Appeler l'API d'OpenAI
response = openai.ChatCompletion.create(
    model="gpt-4o",  # Utilisation du modèle gpt-4o
    messages=[
        {"role": "system", "content": "Tu es un assistant specialisé dans la recherche d'information à partir de documents fournis. Tes réponses doivent absolument provenir du contexte fourni."},
        {"role": "user", "content": prompt}
    ],
    temperature=0
)

# Extraire et afficher la réponse générée
generated_response = response['choices'][0]['message']['content'].strip()
print(f"Réponse générée : {generated_response}")
     

Réponse générée : Le contexte ne fournit pas explicitement une liste des lieux les plus visités. Cependant, il mentionne plusieurs sites et activités touristiques dans la Métropole Nice Côte d’Azur, tels que :

- Le train des pignes à vapeur
- Les ruines du château
- Les pistes de ski
- Les activités estivales comme la randonnée et le VTT
- Le patrimoine Belle Epoque, les plages et les ports

Ces éléments suggèrent des attractions populaires dans la région.


In [19]:
# Requête utilisateur pour laquelle nous cherchons les segments similaires
query = "Quel sont les lieux les plus visité ?"
query_embedding = model.encode(query)

# Calcul des similarités cosinus entre la requête et les segments
similarities = util.cos_sim(query_embedding, embeddings)

# Sélection des top-N documents les plus similaires
top_n = 10
top_results = similarities.topk(top_n)[1]  # Indices des top-N documents similaires

# Accéder aux indices des résultats top-N
top_n_paragraphs = [doc_list[i] for i in top_results[0]]

# Affichage des top-N segments les plus similaires
for i, para in enumerate(top_n_paragraphs, 1):
    print(f"Segment {i} similaire : {para}\n")

Segment 1 similaire : + + Néanmoins, | 
Voirie et trafic | 0 | et
.
Bâti | - | du chalet
Foncier | Acquisition de de
- emprises partielles
réaliser les travaux | ux emprises totales et de deux
de parcelles privées afin de pouvoir
.
Réseaux | 0 | 
Activités | Le projet, en améli
proposées sur le Fr
+ + touristique
pérennisation des a
commerciales de la | orant le fonctionnement des activités
ont de neige participe à attrait
et ainsi à la
ctivités économiques, touristiques et
station.
Paysage | L
+

Segment 2 similaire : St-André la Roche 39 St-Étienne-de-Tinée qui, en toute saison, frappent nos visiteurs et inspirent les plus grands artistes.
25 St-Blaise St-Martin-Vésubie
Bienvenue dans notre Métropole Nice Côte d’Azur ! Nos équipes des bureaux
d’informations touristiques vous attendent !
Saint-Jeannet 40 St-Sauveur sur tinée
26 St-Martin-du-Var Tournefort
Tourrette-Levens 41 Valdeblore
Bienvenue dans notre Métropole Nice Côte d’Azur !
27 Utelle Venanson
Vence
42 Infos pratiques Christ